In [1]:
# tf & keras & VGG19 libs
import tensorflow as tf
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras import Sequential
from tensorflow.keras.applications.vgg19 import preprocess_input

In [ ]:
# Load train and test set
train_dir = '../input/face-mask-12k-images-dataset/Face Mask Dataset/Train'
test_dir = '../input/face-mask-12k-images-dataset/Face Mask Dataset/Test'
val_dir = '../input/face-mask-12k-images-dataset/Face Mask Dataset/Validation'

# Image size
IMG_SIZE = (224, 224)

In [2]:
# augmentation
DataGen = ImageDataGenerator(rotation_range=45,
                                       shear_range=0.2,
                                       zoom_range=0.2,
                                       horizontal_flip=True,
                                       vertical_flip=True,
                                       rescale=1/255.0)

Train_data = DataGen.flow_from_directory(train_dir, target_size=(IMG_SIZE), 
                                         color_mode="rgb",
                                         batch_size=200, 
                                         shuffle=True,
                                         class_mode="categorical")

Test_data = DataGen.flow_from_directory(test_dir, target_size=(IMG_SIZE), 
                                         color_mode="rgb",
                                         batch_size=64, 
                                         shuffle=True,
                                         class_mode="categorical")

Validation_data = DataGen.flow_from_directory(val_dir, target_size=(IMG_SIZE), 
                                         color_mode="rgb",
                                         batch_size=64, 
                                         shuffle=True,
                                         class_mode="categorical")

NameError: name 'train_dir' is not defined

In [4]:
# Initialize the model in sequential
model = Sequential()

# Transfer Learning Model VGG19
vgg19 = VGG19(input_shape=(224,224,3), 
                         include_top=False,
                         weights="imagenet")

# make pre trained model into non trainable bcoz its takes much time
for layer in vgg19.layers:
    layer.trainable = False

# add vgg19 model into our sequence model
model.add(vgg19)

# flatten the model
model.add(Flatten())

# Adding dense layers
model.add(Dense(128, activation="relu", kernel_initializer="he_uniform"))

# Adding output layers
model.add(Dense(2, activation='sigmoid', kernel_initializer="glorot_uniform"))

# Finaly compile the model...
model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [5]:
# summary of the model
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg19 (Functional)          (None, 7, 7, 512)         20024384  
                                                                 
 flatten (Flatten)           (None, 25088)             0         
                                                                 
 dense (Dense)               (None, 128)               3211392   
                                                                 
 dense_1 (Dense)             (None, 2)                 258       
                                                                 
Total params: 23,236,034
Trainable params: 3,211,650
Non-trainable params: 20,024,384
_________________________________________________________________


In [32]:
model.load_weights('./mask_nomask_model.h5')

In [30]:
import numpy as np
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg19 import preprocess_input

labels = ['mask', 'no_mask']

image1 = image.load_img('./mask_unmask_dataset/train/no_mask/Denzel_Washington_059.jpg', target_size = (224, 224))


transformedImage = image.img_to_array(image1)
transformedImage = np.expand_dims(transformedImage, axis = 0)
transformedImage = preprocess_input(transformedImage)

prediction = model.predict(transformedImage)
print(prediction)
print(labels[np.argmax(prediction, axis=1)[0]])

1/1 [==============================] - 0s 28ms/step
[[1.000000e+00 9.613314e-23]]
mask


In [ ]:
Model = model.fit(Train_data, validation_data=(Validation_data), epochs=5)

In [ ]:
model.save("anudari-mask-nomask-v2.h5")

In [34]:
import face_recognition
import cv2
import numpy as np
from tensorflow import keras
from tensorflow.keras.preprocessing import image
#grab preprocess from pre-trained model
from tensorflow.keras.applications.vgg19 import preprocess_input
from keras.models import load_model
from keras_vggface import utils

#video_capture = cv2.VideoCapture(0)
video_capture = cv2.VideoCapture("../Dataset/Video1.mp4")




# Initialize some variables
face_locations = []
face_encodings = []
face_names = []
process_this_frame = True
predictionLabel = ''

while True:
    ret, frame = video_capture.read()

    if process_this_frame:
        small_frame = cv2.resize(frame, (0, 0), fx=0.25, fy=0.25)
        rgb_small_frame = small_frame[:, :, ::-1]
        face_locations = face_recognition.face_locations(rgb_small_frame)

        new_frame = cv2.resize(frame, (224,224))
        x = image.img_to_array(new_frame)
        x = np.expand_dims(x, axis=0)
        x = preprocess_input(x)

        prediction = model.predict(x)
        predictionLabel = labels[np.argmax(prediction, axis=1)[0]]
        ###

        

    #process_this_frame = not process_this_frame


    for (top, right, bottom, left) in face_locations:
        top *= 4
        right *= 4
        bottom *= 4
        left *= 4

        cv2.rectangle(frame, (left - 10, top - 50), (right + 10, bottom + 50), (0, 0, 255), 2)
        cv2.rectangle(frame, (left-11, bottom + 50), (right + 11, bottom + 75), (0, 0, 255), cv2.FILLED)
        font = cv2.FONT_HERSHEY_DUPLEX
        cv2.putText(frame, predictionLabel, (left - 6, bottom + 65), font, 0.5, (255, 255, 255), 1)

    cv2.imshow('Video', frame)

    # Hit 'q' on the keyboard to quit!
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

video_capture.release()
cv2.destroyAllWindows()

1/1 [==============================] - 0s 21ms/step
